##### IMPORTAR AS PLANILHAS

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

##### IMPORTANDO PLANILHA

In [ ]:
path_iw = "../../../data/util/os/"

df_iw47 = pd.read_csv(path_iw + "IW47_Executadas_preparado.csv")

df_iw47.columns

In [ ]:
colunas_tipo_atividade = ["TAM", "Txt.breve operação"]

In [ ]:
df_iw47["Data fim"] = pd.to_datetime(df_iw47["Data fim"])
df_iw47["Data inicio"] = pd.to_datetime(df_iw47["Data inicio"])
df_iw47["Local de instalação"] = df_iw47["Local de instalação"].astype(str)

##### CRIANDO DATAFRAME SEM INCLUIR DATAS FINAIS ANTERIORES A DATAS INICIAIS

In [ ]:
# Removendo linhas onde a data final é superior a data inicial
df_sem_linhas_irregulares = df_iw47.loc[df_iw47["Data fim"]
                                        >= df_iw47["Data inicio"]]

num_linhas_removidas = df_iw47.shape[0] - df_sem_linhas_irregulares.shape[0]
print("Linhas removidas:", num_linhas_removidas)

##### FUNÇÃO QUE ADD COLUNA DE HH POR MES E QUANTIDADE DE MESES PASSADOS DURANTE A OS

In [ ]:
# Soma dos meses, em funcao da quantidade de anos que se passaram e meses, entre o inicio e fim da OS
df = df_sem_linhas_irregulares

df["Meses"] = (
    (df["Data fim"].dt.year - df["Data inicio"].dt.year) * 12
    + (df["Data fim"].dt.month - df["Data inicio"].dt.month)
) + 1
df["HH por mes"] = df["HH final"] / df["Meses"]

##### FUNCAO QUE AGRUPA HH POR ANO, MES, LOCAL E TIPO DE ATIVIDADE

In [ ]:
df["Data inicio"] = df["Data inicio"].apply(lambda x: x.replace(day=1))
df["Data fim"] = df["Data fim"].apply(lambda x: x.replace(day=1))

In [ ]:
colunas_a_agrupar = ["Ano Mes", "Local de instalação", "HH total"]
for coluna_tipo_atividade in colunas_tipo_atividade:
    tipos_de_atividade = df[coluna_tipo_atividade].unique()

    tipos_de_atividade = "HH de " + tipos_de_atividade

    colunas_a_agrupar.extend(tipos_de_atividade)

In [ ]:
# criar subselecoes do df escolhendo todas as OP ativas em cada mes em cada LI

df_agrupado = pd.DataFrame(
    columns=colunas_a_agrupar)

# itera sobre todos os LIs para em cada fazer a seleção do alcance de datas onde houveram operações
for local_instalacao in df["Local de instalação"].unique():
    no_local = df[df["Local de instalação"] == local_instalacao]

    data_inicio_min = no_local["Data inicio"].min()
    data_fim_max = no_local["Data fim"].max()
    espaco_de_tempo = pd.date_range(
        start=data_inicio_min, end=data_fim_max, freq="MS")

    # para cada alcance de datas, seleciona quais eram as operações que estavam ativas durante aquele mês
    for ano_mes in espaco_de_tempo:
        ativas = no_local[(no_local["Data inicio"] <= ano_mes)
                          & (no_local["Data fim"] >= ano_mes)]
        if ativas.shape[0] == 0:
            continue

        # cria o objeto que acumula os resultados do agrupamento de informações
        linha_a_adicionar = {
            "Ano Mes": ano_mes,
            "Local de instalação": local_instalacao
        }

        # conta o total de horas trabalhadas para aquele lugar naquele mes
        hh_por_mes_total = ativas["HH por mes"].sum()
        linha_a_adicionar["HH total"] = hh_por_mes_total

        # itera sobre as colunas das quais as categorias tem que ser separadas para contar a quantidade de HH para cada categoria
        # importante: para cada coluna em colunas_tipo_atividade, a soma de todas as colunas de HH é multiplicada mais uma vez. Como tem 2, o total é o triplo.
        for coluna_tipo_atividade in colunas_tipo_atividade:
            # para cada categoria existente na coluna, conta o total de HH daquele mês
            # e adiciona esta informação em uma nova coluna da linha a ser adicionada ao df agrupado
            for atividade in ativas[coluna_tipo_atividade].unique():
                operacoes_da_atividade = ativas[ativas[coluna_tipo_atividade] == atividade]
                hh_da_ativ_por_mes_total = operacoes_da_atividade["HH por mes"].sum(
                )
                linha_a_adicionar["HH de " +
                                  atividade] = hh_da_ativ_por_mes_total

        df_agrupado.loc[len(df_agrupado)] = linha_a_adicionar

##### TRATAMENTO FINAL PARA PODER EXPORTAR A TABELA

In [ ]:
df_agrupado = df_agrupado.sort_values(
    by=["Ano Mes", "Local de instalação"]).reset_index(drop=True)
df_agrupado['Ano Mes'] = df_agrupado['Ano Mes'].astype(str).str[:7]

df_agrupado.fillna(0, inplace=True)

df_agrupado['Ano Mes'].unique()

##### EXPORTANDO PLANILHA

In [ ]:
df_agrupado.to_csv(path_iw + "os_entrada.csv", index=False)